In [1]:
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
import json 
import os
import pickle
import torch

def loadData(test = False):

    with open('quora_raw_train.json') as f:
        training_examples = json.load(f)
        
    with open('quora_raw_val.json') as f:
        val_examples = json.load(f)

    with open('quora_data_prepro.json') as f:
        vocab = json.load(f)['ix_to_word'] # Vocab = {1: 'the', 2: 'sauce' .....}
        vocab_inv = {word:number for number,word in list(vocab.items())} # Inverted dictionary created          Vocab_inv = {'the':1, 'sauce':2, .....}

    X_train_prelim = [training_examples[i]['question1'] for i in range(len(training_examples))] # Training examples before preprocessing
    Y_train_prelim = [training_examples[i]['question'] for i in range(len(training_examples))]

    X_val_prelim = [val_examples[i]['question1'] for i in range(len(val_examples))]
    Y_val_prelim = [val_examples[i]['question'] for i in range(len(val_examples))]

    if test:
        X_train_prelim, Y_train_prelim, X_val_prelim, Y_val_prelim = X_train_prelim[:1000] , Y_train_prelim[:1000] , X_val_prelim[:1000] , Y_val_prelim[:1000] 

    X_train = []
    Y_train = []

    filename = "../glove.6B.100d.txt"

    lines = open(filename).readlines()
    print(len(lines))
    embeddings = {}
    a = 0
    for line in lines:
        a +=1
        print(a)
        word = line.split()[0]
        embedding = list(map(float, line.split()[1:]))
        if word in vocab.values():
            embeddings[int(vocab_inv[word])] = embedding # This is creating a dictionary with indexes corresponding to the position of the particular word in the vocabulary
            # embeddings = {1:embedding of 'the', 2: embedding of 'sauce'....}
        
    embedded_words = embeddings
    
    # This is basically tokenising each sentence, and converting it into a list of word embeddings. X corresponds to a training example and Y corresponds to its paraphrase
    for i in range(len(X_train_prelim)): 
        print(i)
        sentence_x = X_train_prelim[i]
        sentence_y = Y_train_prelim[i]
        tokensx = word_tokenize(sentence_x)
        tokensy = word_tokenize(sentence_y)
        vectorx = []
        vectory = []
        for word in tokensx:
            try:
                vectorx.append(embedded_words[int(vocab_inv[word])])
            except:
        #If the word is not in the embeddings given, then add a random vector (might have to do something about this :) ) (Potentially could write a spell checker to find misspellings)
                vectorx.append(-2*torch.rand(100,1)) #
        for word in tokensy:
            try:
                vectorx.append(embedded_words[int(vocab_inv[word])])
            except:
                vectory.append(-2*torch.rand(100,1))

        X_train.append(vectorx)
        Y_train.append(vectory)

    return embedded_words, vocab

In [2]:
import pickle
import json

def loadDataPre():
    with open("embeddings", "rb") as f:
        embeddings = pickle.load(f)
    
    with open('quora_data_prepro.json') as f:
        vocab = json.load(f)['ix_to_word'] # Vocab = {1: 'the', 2: 'sauce' .....}
        vocab_inv = {word:number for number,word in list(vocab.items())} # Inverted dictionary created          Vocab_inv = {'the':1, 'sauce':2, .....}
    
    return embeddings, vocab

In [18]:
import Decoder
import Encoder

input_size = 100
embedding_dims = 100
hidden_size_enc = 128
hidden_size_dec = 128
num_layers = 1
vocab_size = len(vocab)
dropout = 0.2
max_sentence_length = 16

print("Embedded words found")

encoder = Encoder.Encoder(max_sentence_length, input_size,hidden_size_enc,dropout,vocab_size,num_layers, 128)
decoder = Decoder.Decoder(encoder, embedding_dims, hidden_size_dec)

Embedded words found


In [19]:
### Implement combined forwards

X_train = Encoder.X_train
Y_train = Encoder.Y_train

decoder.forward(X_train[:,:128], Y_train[:,:128])

0


RuntimeError: 1D tensors expected, got 1D, 0D tensors at /pytorch/aten/src/TH/generic/THTensorEvenMoreMath.cpp:431

In [16]:
[sent,embed]

torch.Size([16, 128])